In [114]:
# Seleniumのインポート
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_binary # ChromeDriverのバイナリを自動でダウンロード・パスを通す
# BeautifulSoupのインポート
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [115]:
# Chromeドライバーの設定
# ChromeDriverのオプションを設定
# ヘッドレスモードを有効化して画面を表示しない
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')  # ヘッドレスモードを追加

driver = webdriver.Chrome(options=options)


In [116]:
try:
    # スクレイピングしたいWebページのURLを指定 
    url = "https://exp-t.jp/account/login/expa"  # スクレイピング対象のURLに変更してください
    driver.get(url)
    
    # ページの読み込みを待機
    wait = WebDriverWait(driver, 10)

except Exception as e:
    print(f"エラーが発生しました: {e}")

In [117]:
# ログイン情報の入力
ID = "sekaino.hiroshi34@gmail.com"
PASSWORD = "h31503150h"

try:
    # IDの入力
    # id="loginId" の要素が見つかるまで待機
    id_element = wait.until(EC.presence_of_element_located((By.ID, "MasterCustomerMail")))
    id_element.send_keys(ID)  # 実際のIDに変更してください
    time.sleep(1)

    # パスワードの入力 
    # id="password" の要素が見つかるまで待機
    password_element = wait.until(EC.presence_of_element_located((By.ID, "MasterCustomerPassword")))
    password_element.send_keys(PASSWORD)  # 実際のパスワードに変更してください
    
    time.sleep(1)
    print("IDの入力が完了しました")

    # ログインボタンのクリック
    login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#LoginForm > div.user-login-btn.mb16 > button")))
    login_button.click()

    time.sleep(1)

except Exception as e:
    print(f"ログイン情報の入力中にエラーが発生しました: {e}")


IDの入力が完了しました


In [118]:
driver.get("https://exp-t.jp/e/event/calendar")

In [119]:
result_df = pd.DataFrame()

# ページのHTMLを取得
html = driver.page_source

# BeautifulSoup4で解析
soup = BeautifulSoup(html, 'html.parser')
schedules = soup.find(class_="mb30")
tables = schedules.find_all('table')

In [120]:
for table in tqdm(tables):
    try:
        elements = table.find_all(class_='fw-b')
        print(elements)
        date = elements[-2].get_text()
        day = date.split('(')[0]
        time = date.split(' ')[1].split('｜')[0]
        count = elements[-1].get_text()
        # day, time, countをDataFrameに変換
        df = pd.DataFrame([[day, time, count]], columns=['day', 'time', 'count'])
        result_df = pd.concat([result_df, df], ignore_index=True)
    except Exception as e:
        print(e)
        break

result_df

 50%|█████     | 7/14 [00:00<00:00, 1947.73it/s]

[<span class="c-yellow fw-b f11"> / 入金済</span>, <span class="fw-b">07/10<span>(木)</span> 12:00｜</span>, <span class="fw-b">0</span>]
[<span class="fw-b">07/10<span>(木)</span> 20:00｜</span>, <span class="fw-b">1</span>]
[<span class="fw-b">07/12<span class="c-blue">(土)</span> 12:00｜</span>, <span class="fw-b">1</span>]
[<span class="fw-b">07/13<span class="c-l-red">(日)</span> 12:00｜</span>, <span class="fw-b">1</span>]
[<span class="fw-b">07/13<span class="c-l-red">(日)</span> 18:00｜</span>, <span class="fw-b">0</span>]
[<span class="fw-b">07/15<span>(火)</span> 12:00｜</span>, <span class="fw-b">0</span>]
[<span class="fw-b">07/15<span>(火)</span> 20:00｜</span>, <span class="fw-b">2</span>]
[<span class="fw-b">0</span>]
list index out of range


,day,time,count
0,07/10,12:00,0
1,07/10,20:00,1
2,07/12,12:00,1
3,07/13,12:00,1
4,07/13,18:00,0
5,07/15,12:00,0
6,07/15,20:00,2


In [121]:
result_df

,day,time,count
0,07/10,12:00,0
1,07/10,20:00,1
2,07/12,12:00,1
3,07/13,12:00,1
4,07/13,18:00,0
5,07/15,12:00,0
6,07/15,20:00,2
